In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test").getOrCreate()

24/01/08 23:04:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
from pyspark.sql import Row
from datetime import datetime

# Create a list of Row objects
stadium_data = [
    Row(id=1, visit_date=datetime(2017, 1, 1), people=10),
    Row(id=2, visit_date=datetime(2017, 1, 2), people=109),
    Row(id=3, visit_date=datetime(2017, 1, 3), people=150),
    Row(id=4, visit_date=datetime(2017, 1, 4), people=99),
    Row(id=5, visit_date=datetime(2017, 1, 5), people=145),
    Row(id=6, visit_date=datetime(2017, 1, 6), people=1455),
    Row(id=7, visit_date=datetime(2017, 1, 7), people=199),
    Row(id=8, visit_date=datetime(2017, 1, 9), people=188)
]

# Create a DataFrame from the list of Row objects
stadium_df = spark.createDataFrame(stadium_data)
from pyspark.sql.functions import col, date_format
stadium_df = stadium_df.withColumn("visit_date", date_format(col("visit_date"), "yyyy-MM-dd"))

# Show the DataFrame
stadium_df.show()

print(stadium_df.dtypes)

+---+----------+------+
| id|visit_date|people|
+---+----------+------+
|  1|2017-01-01|    10|
|  2|2017-01-02|   109|
|  3|2017-01-03|   150|
|  4|2017-01-04|    99|
|  5|2017-01-05|   145|
|  6|2017-01-06|  1455|
|  7|2017-01-07|   199|
|  8|2017-01-09|   188|
+---+----------+------+

[('id', 'bigint'), ('visit_date', 'string'), ('people', 'bigint')]


In [9]:
stadium_df.createOrReplaceTempView("Stadium")

In [11]:
result = spark.sql(
    """          
    select *
    from Stadium s1, Stadium s2, Stadium s3
    where s1.people >= 100 and s2.people >= 100 and s3.people >= 100 

    """
)
result.show()

+---+----------+------+---+----------+------+---+----------+------+
| id|visit_date|people| id|visit_date|people| id|visit_date|people|
+---+----------+------+---+----------+------+---+----------+------+
|  2|2017-01-02|   109|  2|2017-01-02|   109|  2|2017-01-02|   109|
|  2|2017-01-02|   109|  2|2017-01-02|   109|  3|2017-01-03|   150|
|  2|2017-01-02|   109|  2|2017-01-02|   109|  5|2017-01-05|   145|
|  2|2017-01-02|   109|  2|2017-01-02|   109|  6|2017-01-06|  1455|
|  2|2017-01-02|   109|  2|2017-01-02|   109|  7|2017-01-07|   199|
|  2|2017-01-02|   109|  2|2017-01-02|   109|  8|2017-01-09|   188|
|  2|2017-01-02|   109|  3|2017-01-03|   150|  2|2017-01-02|   109|
|  2|2017-01-02|   109|  3|2017-01-03|   150|  3|2017-01-03|   150|
|  2|2017-01-02|   109|  3|2017-01-03|   150|  5|2017-01-05|   145|
|  2|2017-01-02|   109|  3|2017-01-03|   150|  6|2017-01-06|  1455|
|  2|2017-01-02|   109|  3|2017-01-03|   150|  7|2017-01-07|   199|
|  2|2017-01-02|   109|  3|2017-01-03|   150|  8

In [13]:
result = spark.sql(
    """          
    select *
    from Stadium s1, Stadium s2, Stadium s3
    where s1.people >= 100 and s2.people >= 100 and s3.people >= 100 and (
        (s1.id - s2.id = 1 and s2.id - s3.id = 1) or
        (s1.id - s2.id = 1 and s3.id - s1.id = 1) or
        (s2.id - s1.id = 1 and s3.id - s2.id = 1)
    )

    """
)
result.show()

+---+----------+------+---+----------+------+---+----------+------+
| id|visit_date|people| id|visit_date|people| id|visit_date|people|
+---+----------+------+---+----------+------+---+----------+------+
|  5|2017-01-05|   145|  6|2017-01-06|  1455|  7|2017-01-07|   199|
|  6|2017-01-06|  1455|  5|2017-01-05|   145|  7|2017-01-07|   199|
|  6|2017-01-06|  1455|  7|2017-01-07|   199|  8|2017-01-09|   188|
|  7|2017-01-07|   199|  6|2017-01-06|  1455|  5|2017-01-05|   145|
|  7|2017-01-07|   199|  6|2017-01-06|  1455|  8|2017-01-09|   188|
|  8|2017-01-09|   188|  7|2017-01-07|   199|  6|2017-01-06|  1455|
+---+----------+------+---+----------+------+---+----------+------+



In [14]:
result = spark.sql(
    """          
    select distinct s1.*
    from Stadium s1, Stadium s2, Stadium s3
    where s1.people >= 100 and s2.people >= 100 and s3.people >= 100 and (
        (s1.id - s2.id = 1 and s2.id - s3.id = 1) or
        (s1.id - s2.id = 1 and s3.id - s1.id = 1) or
        (s2.id - s1.id = 1 and s3.id - s2.id = 1)
    )

    """
)
result.show()

+---+----------+------+
| id|visit_date|people|
+---+----------+------+
|  5|2017-01-05|   145|
|  6|2017-01-06|  1455|
|  7|2017-01-07|   199|
|  8|2017-01-09|   188|
+---+----------+------+

